In [1]:
from torchvision.datasets import FashionMNIST
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch
import numpy as np
import random

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = 'cpu'
generator = torch.Generator()

# 设置随机种子，确保实验可重复性
torch.random.manual_seed(420)
torch.cuda.manual_seed(420)
generator.manual_seed(420)
np.random.seed(420)
random.seed(420)

# 数据预处理，将FashionMNIST的28x28图像调整到299x299
transform = transforms.Compose([
    transforms.Resize(299),  # 缩放图像使其变为299x299
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # 归一化处理
])


# 从"./dataset/"目录加载FashionMNIST数据集，如果没有则会自动下载。
train_data = FashionMNIST(root='./dataset/', train=True,  download=True,transform=transform)
test_data = FashionMNIST(root='./dataset/', train=False,  download=True,transform=transform)
train_batch = DataLoader(dataset=train_data, batch_size=32,  shuffle=True, num_workers=0, drop_last=False, generator=generator)
test_batch = DataLoader(dataset=test_data, batch_size=32,  shuffle=False, num_workers=0, drop_last=False, generator=generator)

D:\Python\Python39\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import Inception3

class BasicConv2d(nn.Module):

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        **kwargs
    ) -> None:
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return F.relu(x, inplace=True)

# 实现GoogleNet模型
class Model(Inception3):
    def __init__(self, num_classes=10):
        super(Model, self).__init__()
        self.Conv2d_1a_3x3 = BasicConv2d(1, 32, kernel_size=3, stride=2)

    def forward(self, x):
        # N x 3 x 299 x 299
        x = self.Conv2d_1a_3x3(x)
        # N x 32 x 149 x 149
        x = self.Conv2d_2a_3x3(x)
        # N x 32 x 147 x 147
        x = self.Conv2d_2b_3x3(x)
        # N x 64 x 147 x 147
        x = self.maxpool1(x)
        # N x 64 x 73 x 73
        x = self.Conv2d_3b_1x1(x)
        # N x 80 x 73 x 73
        x = self.Conv2d_4a_3x3(x)
        # N x 192 x 71 x 71
        x = self.maxpool2(x)
        # N x 192 x 35 x 35
        x = self.Mixed_5b(x)
        # N x 256 x 35 x 35
        x = self.Mixed_5c(x)
        # N x 288 x 35 x 35
        x = self.Mixed_5d(x)
        # N x 288 x 35 x 35
        x = self.Mixed_6a(x)
        # N x 768 x 17 x 17
        x = self.Mixed_6b(x)
        # N x 768 x 17 x 17
        x = self.Mixed_6c(x)
        # N x 768 x 17 x 17
        x = self.Mixed_6d(x)
        # N x 768 x 17 x 17
        x = self.Mixed_6e(x)
        # N x 768 x 17 x 17
       
        # N x 768 x 17 x 17
        x = self.Mixed_7a(x)
        # N x 1280 x 8 x 8
        x = self.Mixed_7b(x)
        # N x 2048 x 8 x 8
        x = self.Mixed_7c(x)
        # N x 2048 x 8 x 8
        # Adaptive average pooling
        x = self.avgpool(x)
        # N x 2048 x 1 x 1
        x = self.dropout(x)
        # N x 2048 x 1 x 1
        x = torch.flatten(x, 1)
        # N x 2048
        x = self.fc(x)
        # N x 1000 (num_classes)
        return x


In [4]:
from torch.optim import Adam
from torch.nn import functional as F

# 初始化一个模型，输入图片通道数为1，输出特征为10
model = Model(num_classes=10).to(device)
# 使用负对数似然损失函数
criterion = torch.nn.CrossEntropyLoss()
# 初始化Adam优化器，设定学习率为0.005
opt = Adam(model.parameters(), lr=0.001)

D:\Python\Python39\lib\site-packages\torchvision\models\inception.py:80: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn('The default weight initialization of inception_v3 will be changed in future releases of '


In [ ]:
# 进行9次迭代
for _ in range(3):
    # 遍历数据批次
    for n_, batch in enumerate(train_batch):
        # 将输入数据X调整形状并输入到模型
        X = batch[0].to(device)
        # y为真实标签
        y = batch[1].to(device)

        # 前向传播，获取模型输出
        sigma = model.forward(X)
        # 计算损失
        loss = criterion(sigma, y)
        # 计算预测的标签
        y_hat = torch.max(sigma, dim=1)[1]
        # 计算预测正确的数量
        correct_count = torch.sum(y_hat == y)
        # 计算准确率
        accuracy = correct_count / len(y) * 100
        # 反向传播，计算梯度
        loss.backward()
        # 更新模型参数
        opt.step()
        # 清除之前的梯度
        model.zero_grad()
        if n_ % 10 == 0:
            # 打印当前批次的损失和准确率
            print(n_, 'loss:', loss.item(), 'accuracy:', accuracy.item())

D:\Python\Python39\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


0 loss: 10.718929290771484 accuracy: 0.0
10 loss: 3.2805380821228027 accuracy: 37.5
20 loss: 1.7815992832183838 accuracy: 43.75
30 loss: 1.8455778360366821 accuracy: 53.125
40 loss: 1.4085487127304077 accuracy: 62.5
50 loss: 1.472427248954773 accuracy: 56.25
60 loss: 1.7953808307647705 accuracy: 46.875
70 loss: 1.1278141736984253 accuracy: 68.75
80 loss: 1.4390803575515747 accuracy: 56.25
90 loss: 0.8133876919746399 accuracy: 68.75
100 loss: 1.5607022047042847 accuracy: 65.625
110 loss: 0.888356626033783 accuracy: 71.875
120 loss: 0.8314771056175232 accuracy: 62.5
130 loss: 1.2128280401229858 accuracy: 68.75
140 loss: 0.7416073083877563 accuracy: 71.875
150 loss: 0.561935305595398 accuracy: 78.125
160 loss: 0.6898165345191956 accuracy: 68.75
170 loss: 0.6605976223945618 accuracy: 87.5
180 loss: 0.7485594153404236 accuracy: 68.75
190 loss: 0.8794205784797668 accuracy: 62.5
200 loss: 0.5030383467674255 accuracy: 84.375
210 loss: 0.9778007864952087 accuracy: 56.25
220 loss: 0.633854389190

In [ ]:
correct_count = 0
for batch in test_batch:
    test_X = batch[0].to(device)
    test_y = batch[1].to(device)
    sigma = model.forward(torch.tensor(test_X, dtype=torch.float32))
    y_hat = torch.max(sigma, dim=1)[1]
    correct_count += torch.sum(y_hat == test_y)
    
accuracy = correct_count / 10000 * 100
print('accuracy:', accuracy.item())